# Aufgabe - Impfdaten
Kurz nach dem Bekanntwerden des Virus haben die ersten Länder auf Erde-14 mit der Impfstoffentwicklung begonnen, da allen klar war, dass das Virus nur mit einer Herdenimmunität, beispielsweise einer Impfung, kontrollieren werden kann. Ende des Jahres 2052 waren die ersten Impfstoffe bereit für die Bevölkerung. Die ersten drei zugelassenen Impfstoffe sind von den Unternehmen Bioneca, Moderntech und Asterna. Da das Land Dendenland eine ähnliche Gesellschaft zu Deutschland aufweist, werden Sie zusätzlich beauftragt den Impfablauf auszuwerten, um aus möglichen Fehlern zu lernen. Dazu wird Ihnen der folgende Datensatz bereitgestellt:

In [1]:
%use krangl(0.17)
val datenRaw = DataFrame.readCSV("https://raw.githubusercontent.com/Limatationz/kotlin-mooc/main/Abschnitt2/daten/impfzahlenDendenland.csv") //Liest die csv-Datei und wandelt sie in einen DataFrame um
var daten = datenRaw.filterByRow { 
    (!(it["metric"]as String).contains("erst") && !(it["metric"]as String).contains("voll") && !(it["metric"]as String).contains("impfstelle")) || it["metric"] == "personen_erst_kumulativ" || it["metric"] == "personen_voll_kumulativ"
}

Der Aufbau der Daten ist folgendermaßen:

In [6]:
daten.schema()

Name,Type,Values,date,[Str],"2052-12-27, 2052-12-27, 2052-12-27, 2052-12-27, 2052-12-27, 2052-12-27, 2052-12-27, 2052-12-27, 2052...",publication_date,[Str],"2052-12-28 16:15, 2052-12-28 16:15, 2052-12-28 16:15, 2052-12-28 16:15, 2052-12-28 16:15, 2052-12-28...",region,[Str],"BW, BW, BW, BW, BW, BW, BW, BW, BW, BW, BW, BW, BW, BW, BW, BY, BY, BY, BY, BY, BY, BY, BY, BY, BY, ...",metric,[Str],"dosen_kumulativ, personen_erst_kumulativ, personen_voll_kumulativ, dosen_bioneca_kumulativ, dosen_mo...",value,[Str],"2084, 2084, 0, 2084, 0, 0, 624, 964, 151, 0, , 0.018774108378495392, 0.18774108378495394, , , 3389, ..."


In [7]:
daten.head(20)

date,publication_date,region,metric,value
2052-12-27,2052-12-28 16:15,BW,dosen_kumulativ,2084
2052-12-27,2052-12-28 16:15,BW,personen_erst_kumulativ,2084
2052-12-27,2052-12-28 16:15,BW,personen_voll_kumulativ,0
2052-12-27,2052-12-28 16:15,BW,dosen_bioneca_kumulativ,2084
2052-12-27,2052-12-28 16:15,BW,dosen_moderntech_kumulativ,0
2052-12-27,2052-12-28 16:15,BW,dosen_asterna_kumulativ,0


**Hinweis**: In diesem Übungsblatt wird wieder die Bibliothek _krangl_ verwendet. Beispiele finden Sie entweder in der Theorie oder in dem [offiziellen Repository](https://github.com/holgerbrandl/krangl#examples). Auf die Dokumentation können Sie [hier](http://holgerbrandl.github.io/krangl/javadoc/krangl/krangl/index.html) zugreifen.

## Die Daten kennenlernen
In dieser Aufgabe sollen die Daten und deren Struktur kennengelernt werden. Es ist zu erkennen, dass in der Spalte *metric* eine Liste an Beschreibungen für die Daten, die in *value* zu finden sein, vorliegt. Implementieren Sie eine Erweiterungsmethode `gibBeschreibungen()` der Klasse `DataFrame`, die alle Beschreibungen in einer Liste zurückgibt.  
Beispiel: [dosen_kumulativ, personen_erst_kumulativ, personen_voll_kumulativ, ...]

In [8]:
fun DataFrame.gibBeschreibungen(): List<String>{
    // BEGIN SOLUTION
    return this.rows.groupBy { it["metric"] as String }.keys.toList()
    // return (this.cols.get(3).toStrings() as Array<String>).toSet().toList() | Alternative Lösung
    // END SOLUTION
}

In [9]:
val ergebnis = listOf("dosen_kumulativ", "personen_erst_kumulativ", "personen_voll_kumulativ", "dosen_bioneca_kumulativ", "dosen_moderntech_kumulativ", "dosen_asterna_kumulativ", "indikation_alter_dosen", "indikation_beruf_dosen", "indikation_medizinisch_dosen", "indikation_pflegeheim_dosen", "dosen_differenz_zum_vortag", "impf_quote_dosen", "impf_inzidenz_dosen", "dosen_kumulativ_alter_unter60", "dosen_kumulativ_alter_60plus")
if(daten.gibBeschreibungen() != ergebnis)
    throw AssertionError("daten.gibBeschreibungen(): ${daten.gibBeschreibungen()} != ${ergebnis}")

## Wieviele Dosen wurden an einem bestimmten Tag verimpft?
Implementieren Sie eine Erweiterungsmethode `gibAnzahlDosen(datum, region)` der Klasse `DataFrame`, die die Anzahl der Impfdosen, welche an dem übergebenen Datum und in der übergebenen Region verimpft wurden, zurückgibt. Zur Vereinfachung ist anzunehmen, dass das Datum als `String` im Format `yyyy-mm-dd` und die Region als `String` mit zwei Buchstaben übergeben werden. In dieser Aufgabe soll nicht mit dem Eintrag `dosen_differenz_zum_vortag` gearbeitet werden, da die Daten unvollständig sind (näheres dazu in der nächsten Aufgabe). Ist das Datum nicht in den Daten vorhanden, soll `-1` zurückgegeben werden.

In [10]:
fun DataFrame.gibAnzahlDosen(datum: String, region: String): Int {
    // BEGIN SOLUTION 
    val dosenKumulativ = daten.filterByRow { it["metric"] == "dosen_kumulativ" && it["region"] == region }.addRowNumber()
    val rowNumber = dosenKumulativ.rows.find { it["date"] == datum }?.get("row_number")
    val rowGestern = (rowNumber as Int?)?.minus(1)
    if (rowGestern == 0){
        return (dosenKumulativ.rows.find { it["row_number"] == rowNumber }?.get("value") as String).toInt()
    }
    else {
        val dosenHeute = dosenKumulativ.rows.find { it["row_number"] == rowNumber }?.get("value") as String?
        val dosenGestern = dosenKumulativ.rows.find { it["row_number"] == rowGestern }?.get("value") as String?
        if (dosenHeute != null && dosenGestern != null){
            return dosenHeute.toInt() - dosenGestern.toInt()
        }
        return -1
    }
    // END SOLUTION
}

In [11]:
val tests = mapOf("2053-04-09" to 544898, "2052-12-27" to 21566, "2053-01-18" to 74406, "2054-03-05" to -1, "2050-01-03" to -1, "2054-23-0" to -1)
for ((datum, anzahl) in tests){
    if (daten.gibAnzahlDosen(datum, "DE") != anzahl)
        throw AssertionError("daten.gibAnzahlDosen($datum): ${daten.gibAnzahlDosen(datum, "DE")} != ${anzahl}")
}

## Verbesserung der Daten I
Mit der Methode `gibAnzahlDosen(datum, region)` kann nun die Differenz der Dosen zum Vortag ausgerechnet werden. Da der Datensatz einige Lücken bezüglich des Eintrags `dosen_differenz_zum_vortag` aufweist, sollen diese jetzt vefüllt werden. Dazu bauen wir uns einen neuen Datensatz `datenVollstaendig` in der Funktion `vervollstaenigeLuecken(daten)` zusammen. Das Grundgerüst des `DataFrame`s ist bereits vorbereitet. Eine Zeile kann mit der Methode [`bindRows(map)`](https://holgerbrandl.github.io/krangl/javadoc/krangl/krangl/bind-rows.html) angehängt werden.

In [12]:
fun vervollstaendigeLuecken(daten: DataFrame): DataFrame {
    var datenVollstaendig: DataFrame = emptyDataFrame()
    // BEGIN SOLUTION 
    daten.rows.forEach {
        if (it["metric"] != "dosen_differenz_zum_vortag"){
            datenVollstaendig = datenVollstaendig.bindRows(
                mapOf("date" to it["date"], "publication_date" to it["publication_date"], "region" to it["region"], "metric" to it["metric"], "value" to it["value"])
            ) 
        }
        else{
            val diff: String = daten.gibAnzahlDosen(it["date"] as String, it["region"] as String).toString()
            datenVollstaendig = datenVollstaendig.bindRows(
                mapOf("date" to it["date"], "publication_date" to it["publication_date"], "region" to it["region"], "metric" to it["metric"], "value" to diff)
            )
        }

    }
    // END SOLUTION
    return datenVollstaendig
}

In [13]:
//Hinweis: dieser Test kann eine längere Zeit als gewohnt benötigen, da der komplette Datensatz durchlaufen wird.
val testdaten = vervollstaendigeLuecken(daten)
val luecken = (testdaten.filterByRow { it["metric"] == "dosen_differenz_zum_vortag" && it["value"] == "" }.count().get(0).get(0) as Int)
if (luecken > 0)
    throw AssertionError("Anzahl der Lücken: ${luecken} > 0")
if(daten.nrow != testdaten.nrow)
    throw AssertionError("Es wurden nicht alle Zeilen des Datensatzes übernommen: ${daten.nrow} != ${testdaten.nrow}")
daten = testdaten

## Verbesserung der Daten II
Bei genaueren Betrachtung der Daten mit einem genaueren Blick auf die Datentypen der Spalten, fällt auf, dass alle Werte der Spalte *value* den Datentyp `String` besitzen. Da aber in dieser Spalte nur Fließkommazahlen oder nichts vorzufinden ist, würde der Datenyp `Double` besser passen. Passen Sie Ihre Lösung von *Verbesserung der Daten I* insoweit an, dass in der Spalte *value* nur noch Werte des Datentyps `Double` zu finden sind, falls ein gültiger Wert enthalten ist. Ansonsten soll `null` eingefügt werden. Schreiben Sie die Lösung zu dieser Aufgabe in die Funktion `vervollstaendigeDaten(daten)`.

In [14]:
fun vervollstaendigeDaten(daten: DataFrame): DataFrame {
    var datenVollstaendig: DataFrame = emptyDataFrame()
    // BEGIN SOLUTION 
    daten.rows.forEach {
        if (it["metric"] != "dosen_differenz_zum_vortag"){
            if(it["value"] == ""){
                datenVollstaendig = datenVollstaendig.bindRows(
                    mapOf("date" to it["date"], "publication_date" to it["publication_date"], "region" to it["region"], "metric" to it["metric"], "value" to null)
                )
            }
            else{
                datenVollstaendig = datenVollstaendig.bindRows(
                    mapOf("date" to it["date"], "publication_date" to it["publication_date"], "region" to it["region"], "metric" to it["metric"], "value" to (it["value"] as String).toDouble())
                )
            }  
        }
        else{
            val diff = daten.gibAnzahlDosen(it["date"] as String, it["region"] as String).toDouble()
            datenVollstaendig = datenVollstaendig.bindRows(
                mapOf("date" to it["date"], "publication_date" to it["publication_date"], "region" to it["region"], "metric" to it["metric"], "value" to diff)
            )
        }

    }
    // END SOLUTION
    return datenVollstaendig
}

In [15]:
//Hinweis: dieser Test kann eine längere Zeit als gewohnt benötigen, da der komplette Datensatz durchlaufen wird.
val testdaten = vervollstaendigeDaten(daten)
val luecken = (testdaten.filterByRow { it["metric"] == "dosen_differenz_zum_vortag" && it["value"] == null}.count().get(0).get(0) as Int)
if (luecken > 0)
    throw AssertionError("Anzahl der Lücken: ${luecken} > 0")
if (testdaten.get("value")::class.simpleName != "DoubleCol")
    throw AssertionError("Datentyp der Spalte value: ${testdaten.get("value")::class.simpleName} != DoubleCol")
if(daten.nrow != testdaten.nrow)
    throw AssertionError("Es wurden nicht alle Zeilen des Datensatzes übernommen: ${daten.nrow} != ${testdaten.nrow}")
daten = testdaten    

## Wann wurden am meisten Dosen mit der Indikaton Alter verimpft?
In den Daten ist außerdem angegeben aus welchem Grund eine Impfung verabreicht wurde. Implementieren Sie eine Erweiterungsmethode `gibDatumAlter()`, die das Datum des Tages zurückgibt, an dem am meisten Dosen augrund der Indikation Alter verabreicht wurden.

In [16]:
fun DataFrame.gibDatumAlter(): String {
    // BEGIN SOLUTION
    val alter = this.filterByRow { it["metric"] == "indikation_alter_dosen" && it["region"] == "DE" && it["value"] != null }.addRowNumber()
    //Lösung mit reduce
    /*return alter.rows.reduce { zeile1, zeile2 ->
        val anzahlZeile1 = zeile1["value"] as Double
        val anzahlZeile2 = zeile2["value"] as Double
        val dif01 = if (zeile1["row_number"] == 1)
                        anzahlZeile1
                    else{
                        val anzahlZeile0 = alter.row((zeile1["row_number"] as Int)-2)["value"] as Double
                        anzahlZeile1 - anzahlZeile0
                    }
        val dif12 = anzahlZeile2 - anzahlZeile1
        if(dif01 > dif12){
            zeile1
        }
        else{
            zeile2
        }
    }.get("date") as String*/
    
    //Lösung mit maxByOrNull
    return alter.rows.maxByOrNull { 
        val anzahlZeile1 = (it["value"] as Double)
        if (it["row_number"] == 1 || it["row_number"] == 0)
            anzahlZeile1
        else{
            val anzahlZeile0 = (alter.row((it["row_number"] as Int)-2)["value"] as Double)
            anzahlZeile1 - anzahlZeile0
        }
    }?.get("date") as String
    // END SOLUTION
}

In [17]:
if (daten.gibDatumAlter() != "2053-04-05")
    throw AssertionError("testdaten.gibDatumAlter(): ${testdaten.gibDatumAlter()} != 2053-04-05")

## Wer wird eigentlich geimpft?
In dieser Aufgabe soll der Frage nachgegangen werden, wer eigentlich geimpft wird. In den Daten gibt es dazu 4 Metriken: *indikation_alter_dosen*, *indikation_beruf_dosen*, *indikation_medizinisch_dosen*, *indikation_pflegeheim_dosen*. Implementieren Sie eine Erweiterungsmethode `gibIndikationen()` der Klasse `DataFrame`, die eine Liste des Datentyps `List<Map<String, Map<String, Any>>>` zurückgibt. In dieser soll für jedes Datum eine Map, in der die verschiedenenen Indikationen der Schlüssel (Alter, Beruf, Medizinisch, Pflegeheim) sind, existieren. Die Werte zu den Indikationen sind die an diesem Tag verimpften Dosen der Infikation. Außerdem soll unter dem Schlüssel *Max* eine weitere Map zu finden sein, die als Schlüssel die Indikation mit den meisten Dosen an diesem Datum und als Wert die Anzahl besitzt.  
Beispiel:  
[{2052-12-27={Alter=5833.0, Beruf=7618.0, Medizinisch=962.0, Pflegeheim=11646.0, Max={Pflegeheim=11646.0}}}, {2052-12-28={Alter=6399.0, Beruf=7346.0, Medizinisch=699.0, Pflegeheim=10121.0, Max={Pflegeheim=10121.0}}}, ... ]

In [18]:
fun DataFrame.gibIndikationen(): List<Map<String, Map<String, Any>>> {
    // BEGIN SOLUTION
    val filtered = this.filterByRow { 
        (it["metric"] == "indikation_alter_dosen" || it["metric"] == "indikation_beruf_dosen" || 
        it["metric"] == "indikation_medizinisch_dosen" || it["metric"] == "indikation_pflegeheim_dosen" ) 
        && it["region"] == "DE" && it["value"] != null 
    }.addRowNumber()
    return filtered.groupBy("date").groups().map { 
        val alter = it["value"][0] as Double
        val diffAlter: Double = if (it["row_number"][0] as Int > 4){
                            val zeile0 = (it["row_number"][0] as Int) - 5
                            val alter0 = filtered.row(zeile0)["value"] as Double
                            alter - alter0
                        }
                        else{
                            alter
                        }
        val beruf = it["value"][1] as Double
        val diffBeruf: Double = if (it["row_number"][1] as Int > 4){
                            val zeile0 = (it["row_number"][1] as Int) - 5
                            val beruf0 = filtered.row(zeile0)["value"] as Double
                            beruf - beruf0
                        }
                        else{
                            beruf
                        }
        val medi = it["value"][2] as Double
        val diffMedi: Double = if (it["row_number"][2] as Int > 4){
                            val zeile0 = (it["row_number"][2] as Int) - 5
                            val medi0 = filtered.row(zeile0)["value"] as Double
                            medi - medi0
                        }
                        else{
                            medi
                        }
        val pflege = it["value"][3] as Double
        val diffPflege: Double = if (it["row_number"][3] as Int > 4){
                            val zeile0 = (it["row_number"][3] as Int) - 5
                            val pflege0 = filtered.row(zeile0)["value"] as Double
                            pflege - pflege0
                        }
                        else{
                            pflege
                        }
        val max = mapOf(
            when (listOf(diffAlter, diffBeruf, diffMedi, diffPflege).maxOrNull()){
                diffAlter -> "Alter";
                diffBeruf -> "Beruf";
                diffMedi -> "Medizinisch";
                diffPflege -> "Pflegeheim";
                else -> "else"
            }
            to
             listOf(diffAlter, diffBeruf, diffMedi, diffPflege).maxOrNull()
        )
        mapOf(it["date"][0] as String to mapOf("Alter" to diffAlter, "Beruf" to diffBeruf, "Medizinisch" to diffMedi, "Pflegeheim" to diffPflege, "Max" to max))
    }
    // END SOLUTION
}

In [19]:
val testdaten = daten.gibIndikationen()
val testZeilen = listOf("{2052-12-27={Alter=5833.0, Beruf=7618.0, Medizinisch=962.0, Pflegeheim=11646.0, Max={Pflegeheim=11646.0}}}", "{2052-12-28={Alter=6399.0, Beruf=7346.0, Medizinisch=699.0, Pflegeheim=10121.0, Max={Pflegeheim=10121.0}}}")
for (i in 0 until testZeilen.size){
    if (testdaten[i].toString() != testZeilen[i])
        throw AssertionError("${testdaten[i].toString()} != ${testZeilen[i]}")
}

testdaten.forEach {
    val max = it.values.toList()[0].values.take(4).toList().maxByOrNull { it as Double }
    var string = it.values.toList()[0].values.drop(4)[0].toString()
    val (indikator, dosen) = string.substring(1, string.length-1).split("=")
    if(max != dosen.toDouble())
        throw AssertionError("Maximum in Zeile $it: $dosen != $max")
}

## Mit dem neuen Datensatz arbeiten
In der letzten Aufgabe wurde ein neuer Datensatz erstellt, der die Indikation der Impfungen in Verbindung mit den Tagen setzt und das Maximum bereitstellt. Nun sollen Sie eine Funktion höherer Ordnung `analysiereIndikation` implementieren, die diese Daten variabel wieder in seine Einzelteile zerlegen kann. Der Funktion sollen die Daten nach der Verarbeitung mit `gibIndikationen()` übergeben werden. Die Rückgabe soll eine Funktion darstellen, die ein Datum als Zeichenkette und die Indikation (Alter, Beruf, Medizinisch, Pflegeheim, Max) übergeben bekommt. Die Rückgabe stellt den Wert, der der Indikation zugeordnet ist dar (`Any`).  
Falls Sie keine Lösung bei der Implementierung der Erweiterungsmethode `gibIndikationen()` gefunden haben, können Sie den vorbereiteten Datensatz stattdessen verwenden.

Beispielaufruf: `analysiereIndikation(daten.gibIndikationen())("2052-12-27", "Beruf")`, Rückgabe: `7618.0`

In [20]:
// BEGIN SOLUTION
fun analysiereIndikation(daten: List<Map<String, Map<String, Any>>>): (String, String) -> Any = {datum, indikation -> daten.filter { it.keys.first() == datum }.get(0).values.toList().get(0).get(indikation) as Any }
// END SOLUTION

In [21]:
//Wählen Sie aus, welche Daten im Test verwendet werden sollen, indem Sie die entpreschende Zeile auskommentieren
val testdaten = listOf(mapOf("2052-12-27" to mapOf("Alter" to 5833.0, "Beruf" to 7618.0, "Medizinisch" to 962.0, "Pflegeheim" to 11646.0, "Max" to mapOf("Pflegeheim" to 11646.0))), mapOf("2052-12-28" to mapOf("Alter" to 6399.0, "Beruf" to 7346.0, "Medizinisch" to 699.0, "Pflegeheim" to 10121.0, "Max" to mapOf("Pflegeheim" to 10121.0))))
//val testdaten = daten.gibIndikationen()
// BEGIN SOLUTION
// END SOLUTION

In [22]:
val testdaten1 = listOf(mapOf("Alter" to 5833.0), mapOf("Beruf" to 7618.0), mapOf("Medizinisch" to 962.0), mapOf("Pflegeheim" to 11646.0), mapOf("Max" to mapOf("Pflegeheim" to 11646.0)))
for (value in testdaten1){
    for((indikation, wert) in value){
        if(analysiereIndikation(testdaten)("2052-12-27", indikation) != wert)
            throw AssertionError("analysiereIndikation(testdaten)('2052-12-27', '$indikation'): ${analysiereIndikation(testdaten)("2052-12-27", indikation)} != $wert")
    }
}

val testdaten2 = listOf(mapOf("Alter" to 6399.0), mapOf("Beruf" to 7346.0), mapOf("Medizinisch" to 699.0), mapOf("Pflegeheim" to 10121.0), mapOf("Max" to mapOf("Pflegeheim" to 10121.0)))
for (value in testdaten2){
    for((indikation, wert) in value){
        if(analysiereIndikation(testdaten)("2052-12-28", indikation) != wert)
            throw AssertionError("analysiereIndikation(testdaten)('2052-12-28', '$indikation'): ${analysiereIndikation(testdaten)("2052-12-27", indikation)} != $wert")
    }
}